Predict which Tweets are about real disasters and which ones are not

In [40]:
import nltk
import pandas as pd
import string
from nltk.tokenize import word_tokenize

# IMPORT DATA

In [44]:
data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [45]:
train_data = data[['text','target']]
data_real = train_data[train_data['target']==1]
data_not_real = train_data[train_data['target']==0]

# DATA PROCESSING

In [46]:
useless_words = nltk.corpus.stopwords.words('english') + list (string.punctuation)

In [47]:
def tokenize(words):
    return word_tokenize(words)
def build_bag_of_words(words):
    return  {
        word:True for word in words \
        if not word in useless_words}

In [48]:
real = []
for i in data_real['text']:
    
    words = tokenize(i)
    
    # Then we use the function we wrote earlier to create word features in the format nltk expects.
    real.append((build_bag_of_words(words),1))

In [49]:
not_real = []
for i in data_not_real['text']:
    
    words = tokenize(i)
    
    # Then we use the function we wrote earlier to create word features in the format nltk expects.
    not_real.append((build_bag_of_words(words),0))

In [50]:
print(len(real))
real[0]

3271


({'Our': True,
  'Deeds': True,
  'Reason': True,
  'earthquake': True,
  'May': True,
  'ALLAH': True,
  'Forgive': True,
  'us': True},
 1)

# TRAIN THE MODEL

In [51]:
from nltk.classify import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(real + not_real)

# TEST DATA PROCESSING

In [54]:
test = []
for i in test_data['text']:
    
    words = tokenize(i)
    
    # Then we use the function we wrote earlier to create word features in the format nltk expects.
    test.append(build_bag_of_words(words))

In [60]:
prediction = []
for i in range(0,len(test)):
    predictions=classifier.classify(test[i])
    prediction.append(predictions)

In [59]:
len(test)

3263

# CREATE FINAL DATAFRAME

In [79]:
final_labels = pd.DataFrame((prediction),columns={'target'})
final = pd.concat([test_data['id'],final_labels],axis=1,join='inner')
final.to_csv('final.csv', index=False)